In [1]:
import numpy as np
import pandas as pd 

# importa o arquivo e guarda em um dataframe do Pandas
df = pd.read_csv( 'results.csv', sep=',', index_col=None) 
n_alg = 8 #numero de algoritmos
n = 3 * n_alg #numero de testes

df.set_index('Busca', inplace=True)

tiny = df.iloc[0:n].groupby('Busca').mean().round(2)
small = df.iloc[n:2*n].groupby('Busca').mean().round(2)
medium = df.iloc[2*n:3*n].groupby('Busca').mean().round(2)
big = df.iloc[3*n:4*n].groupby('Busca').mean().round(2)

## Tiny Maze

### Tempo e Memória

In [11]:
tiny.iloc[:, 0:2].sort_values(by=['Tempo'])

,Tempo,Maximo tam de Nos na memoria
Busca,,
Greedy Search - Shockwave Heuristic,0.02,15.33
Greedy Search - Manhattan Distance,0.03,17.00
A* Search - Manhattan Distance,0.05,23.33
A* Search - Shockwave Heuristic,0.05,22.00
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,0.06,20.67
DFS,0.07,29.67
BFS,0.10,18.33
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,0.21,20.00


### Nós Expandidos

In [3]:
tiny.iloc[:, 2:].sort_values(by=['Number of expanded nodes'])

,Number of expanded nodes,Number of expanded food nodes,Number of nodes in the path,Number of food nodes in the path
Busca,,,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,116.67,112.67,49.67,49.00
A* Search - Manhattan Distance,109.67,107.33,46.33,45.67
A* Search - Shockwave Heuristic,104.33,101.67,46.33,45.67
BFS,281.33,203.67,37.00,33.00
DFS,170.00,116.00,119.67,76.33
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,156.33,118.33,57.00,40.67
Greedy Search - Manhattan Distance,59.67,42.33,53.00,38.33
Greedy Search - Shockwave Heuristic,36.00,28.67,37.00,29.33


## Small Maze

### Tempo e Memória

In [4]:
small.iloc[:, 0:2].sort_values(by=['Tempo'])

,Tempo,Maximo tam de Nos na memoria
Busca,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,0.58,61.67
A* Search - Manhattan Distance,0.50,48.00
A* Search - Shockwave Heuristic,0.55,46.33
BFS,0.78,22.33
DFS,0.62,104.33
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,0.43,41.00
Greedy Search - Manhattan Distance,0.14,31.33
Greedy Search - Shockwave Heuristic,0.10,34.67


### Nós expandidos

In [5]:
small.iloc[:, 2:].sort_values(by=['Number of expanded nodes'])

,Number of expanded nodes,Number of expanded food nodes,Number of nodes in the path,Number of food nodes in the path
Busca,,,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,673.67,640.67,146.00,145.00
A* Search - Manhattan Distance,537.67,505.00,110.67,109.67
A* Search - Shockwave Heuristic,508.67,478.00,110.67,109.67
BFS,958.00,722.33,94.67,65.67
DFS,711.33,532.00,308.67,227.67
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,513.33,377.33,149.33,102.33
Greedy Search - Manhattan Distance,129.33,98.33,106.67,83.67
Greedy Search - Shockwave Heuristic,93.67,62.00,94.67,63.00


## Medium Maze

### Tempo e Memória

In [6]:
medium.iloc[:, 0:2].sort_values(by=['Tempo'])

,Tempo,Maximo tam de Nos na memoria
Busca,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,5.10,152.67
A* Search - Manhattan Distance,4.76,134.67
A* Search - Shockwave Heuristic,5.51,144.00
BFS,3.21,47.33
DFS,2.79,232.67
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,2.57,73.33
Greedy Search - Manhattan Distance,0.34,50.33
Greedy Search - Shockwave Heuristic,0.19,45.33


### Nós expandidos

In [7]:
medium.iloc[:, 2:].sort_values(by=['Number of expanded nodes'])

,Number of expanded nodes,Number of expanded food nodes,Number of nodes in the path,Number of food nodes in the path
Busca,,,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,2396.67,1986.67,213.33,207.00
A* Search - Manhattan Distance,2194.33,1825.67,155.33,149.00
A* Search - Shockwave Heuristic,2260.33,1895.00,155.33,149.00
BFS,1531.33,1151.33,69.33,41.00
DFS,1271.00,966.00,712.00,556.00
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,1220.67,893.00,95.33,52.33
Greedy Search - Manhattan Distance,163.00,115.67,118.67,92.67
Greedy Search - Shockwave Heuristic,68.33,40.67,69.33,41.33


## Big Maze

### Tempo e Memória

In [8]:
big.iloc[:, 0:2].sort_values(by=['Tempo'])

,Tempo,Maximo tam de Nos na memoria
Busca,,
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,15.72,246.67
A* Search - Manhattan Distance,17.06,256.67
A* Search - Shockwave Heuristic,19.65,268.33
BFS,16.03,73.67
DFS,11.98,614.33
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,14.92,180.00
Greedy Search - Manhattan Distance,4.54,95.00
Greedy Search - Shockwave Heuristic,0.69,71.67


### Nós expandidos

In [10]:
big.iloc[:, 2:].sort_values(by=['Number of expanded nodes'])

,Number of expanded nodes,Number of expanded food nodes,Number of nodes in the path,Number of food nodes in the path
Busca,,,,
BFS,3961.00,3058.67,153.00,112.00
A* Search - Manhattan Distance,3828.33,3226.33,203.00,199.00
A* Search - Shockwave Heuristic,3800.67,3212.33,203.00,199.00
A* Search - Ghost Avoidance Heuristic with Euclidean Distance,3696.33,3001.67,265.00,261.00
Greedy Search - Ghost Avoidance Heuristic with Euclidean Distanc,3645.33,2774.00,400.33,300.33
DFS,3008.67,2324.33,1097.67,849.67
Greedy Search - Manhattan Distance,1089.00,815.00,265.67,172.67
Greedy Search - Shockwave Heuristic,152.00,104.67,153.00,105.67
